# Modelamiento
En este notebook se realizará el entrenamiento y validación de los modelos de riesgo crediticio.

## Cargue de librerías y parámetros

In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import sys
import os

# Agrega la ruta del directorio 'src' al path
sys.path.append(os.path.abspath('../src'))

# Importar los módulos
from procesamiento_datos import *
from modelado import *
from evaluacion import *

Función de procesamiento de datos cargadas correctamente.


In [2]:
repo_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
datos_raw = "..//data/raw/"
datos_processed = "..//data/processed/"

## Cargue de datos preprocesados

In [3]:
# Cargar datos preprocesados
df = cargar_datos(datos_processed+'datos_modelado.csv')

print("Dimensiones del dataset:", df.shape)
print("\nColumnas disponibles:")
print(df.columns.tolist())
print("\nDistribución de la variable objetivo:")
print(df['default'].value_counts(normalize=True) * 100)

Dimensiones del dataset: (2697, 39)

Columnas disponibles:
['NIT', 'Total_facturado', 'Total_por_pagar', 'Total_pagado', 'Cantidad_facturas', 'Dias_mora_promedio', 'Porcentaje_pagos_tiempo', 'Promedio_ratio_pago', 'Desv_dias_demora', 'Maximo_dias_demora', 'Facturas_vencidas', 'Factura_promedio', 'Porcentaje_facturas_vencidas', 'score_credito', 'categoria_riesgo', 'default', 'PERIODO', 'FECHA_CORTE', 'ACTIVOS_CORRIENTES', 'GANANCIAS', 'PATRIMONIO', 'TOTAL_ACTIVOS', 'ACTIVOS_NO_CORRIENTES', 'PASIVOS_NO_CORRIENTES', 'TOTAL_PASIVOS', 'VAR_ACTIVOS_CORRIENTES', 'VAR_GANANCIAS', 'VAR_PATRIMONIO', 'VAR_ACTIVOS', 'VAR_Activos_NO_CORRIENTES', 'VAR_PASIVOS_NO_CORRIENTES', 'VAR_TOTAL_PASIVOS', 'PASIVOS_CORRIENTES', 'VAR_PASIVOS_CORRIENTES', 'UTILIDAD_NETA', 'LIQUIDEZ_CORRIENTE', 'CAPITAL_TRABAJO', 'ENDEUDAMIENTO', 'ROA']

Distribución de la variable objetivo:
default
0    95.068595
1     4.931405
Name: proportion, dtype: float64


## Limpieza de Datos

Antes de escalar los datos, nos aseguraremos de que no haya valores faltantes o infinitos.

# Modelamiento de Riesgo de Default

Este notebook se enfoca en el desarrollo y entrenamiento de modelos predictivos para identificar el riesgo de default en clientes.

## Carga y Preparación de Datos

Cargaremos el dataset procesado y realizaremos la preparación necesaria para el modelamiento.

In [11]:
# Función para verificar datos
def verificar_datos(X, nombre="Dataset"):
    nas = X.isna().sum()
    nas_total = nas.sum()
    inf_count = np.isinf(X.select_dtypes(include=np.number)).sum().sum()
    
    print(f"\nVerificación de {nombre}:")
    if nas_total > 0:
        print("\nColumnas con valores faltantes:")
        print(nas[nas > 0])
    print(f"\nTotal de valores faltantes: {nas_total}")
    print(f"Valores infinitos: {inf_count}")

# Función para limpiar datos
def limpiar_datos(X):
    X_clean = X.copy()
    # Reemplazar infinitos con NaN
    X_clean = X_clean.replace([np.inf, -np.inf], np.nan)
    # Para cada columna, imputar con la mediana si hay valores faltantes
    for col in X_clean.columns:
        if X_clean[col].isna().any():
            mediana = X_clean[col].median()
            X_clean[col] = X_clean[col].fillna(mediana)
    return X_clean

# Verificar estado inicial
print("Estado inicial de los datos:")
verificar_datos(X_train, "X_train")
verificar_datos(X_test, "X_test")

# Limpiar datos
X_train = limpiar_datos(X_train)
X_test = limpiar_datos(X_test)

# Verificar después de la limpieza
print("\nDespués de la limpieza:")
verificar_datos(X_train, "X_train")
verificar_datos(X_test, "X_test")

Estado inicial de los datos:

Verificación de X_train:

Total de valores faltantes: 0
Valores infinitos: 0

Verificación de X_test:

Total de valores faltantes: 0
Valores infinitos: 0

Después de la limpieza:

Verificación de X_train:

Total de valores faltantes: 0
Valores infinitos: 0

Verificación de X_test:

Total de valores faltantes: 0
Valores infinitos: 0


## Escalado de Datos

Ahora que los datos están limpios, procedemos con el escalado.

In [12]:
# Escalar variables numéricas
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

# Obtener columnas numéricas (excluyendo las dummies)
columnas_numericas = X_train.select_dtypes(include=['float64', 'int64']).columns

# Aplicar escalado
X_train_scaled[columnas_numericas] = scaler.fit_transform(X_train[columnas_numericas])
X_test_scaled[columnas_numericas] = scaler.transform(X_test[columnas_numericas])

# Verificar que no haya valores faltantes o infinitos después del escalado
print("Verificación final después del escalado:")
verificar_datos(X_train_scaled, "X_train_scaled")
verificar_datos(X_test_scaled, "X_test_scaled")

# Guardar los datos procesados y limpios
X_train_scaled.to_csv('../data/processed/X_train.csv', index=False)
X_test_scaled.to_csv('../data/processed/X_test.csv', index=False)
y_train.to_csv('../data/processed/y_train.csv', index=False)
y_test.to_csv('../data/processed/y_test.csv', index=False)

print("\nDimensiones finales de los conjuntos de datos:")
print(f"X_train: {X_train_scaled.shape}")
print(f"X_test: {X_test_scaled.shape}")
print(f"y_train: {y_train.shape}")
print(f"y_test: {y_test.shape}")

Verificación final después del escalado:

Verificación de X_train_scaled:

Total de valores faltantes: 0
Valores infinitos: 0

Verificación de X_test_scaled:

Total de valores faltantes: 0
Valores infinitos: 0

Dimensiones finales de los conjuntos de datos:
X_train: (2157, 2716)
X_test: (540, 2716)
y_train: (2157,)
y_test: (540,)


## Limpieza de Datos

Verificaremos y limpiaremos valores faltantes o infinitos en nuestros datos antes del entrenamiento.

In [13]:
# Verificar valores faltantes o infinitos
def verificar_datos(X, nombre="Dataset"):
    print(f"\nVerificación de {nombre}:")
    print(f"Valores faltantes: {X.isna().sum().sum()}")
    print(f"Valores infinitos: {np.isinf(X.select_dtypes(include=np.number)).sum().sum()}")
    
verificar_datos(X_train_scaled, "X_train_scaled")
verificar_datos(X_test_scaled, "X_test_scaled")

# Reemplazar valores infinitos con NaN
X_train_scaled = X_train_scaled.replace([np.inf, -np.inf], np.nan)
X_test_scaled = X_test_scaled.replace([np.inf, -np.inf], np.nan)

# Imputar valores faltantes con la mediana
for col in X_train_scaled.columns:
    if X_train_scaled[col].isna().any():
        mediana = X_train_scaled[col].median()
        X_train_scaled[col] = X_train_scaled[col].fillna(mediana)
        X_test_scaled[col] = X_test_scaled[col].fillna(mediana)

# Verificar después de la limpieza
verificar_datos(X_train_scaled, "X_train_scaled (después de limpieza)")
verificar_datos(X_test_scaled, "X_test_scaled (después de limpieza)")


Verificación de X_train_scaled:
Valores faltantes: 0
Valores infinitos: 0

Verificación de X_test_scaled:
Valores faltantes: 0
Valores infinitos: 0

Verificación de X_train_scaled (después de limpieza):
Valores faltantes: 0
Valores infinitos: 0

Verificación de X_test_scaled (después de limpieza):
Valores faltantes: 0
Valores infinitos: 0


## Entrenamiento de Modelos (con datos limpios)

Ahora entrenaremos los modelos con los datos limpios.

In [9]:
# Importar joblib para guardar los modelos
import joblib

# Crear y entrenar los modelos
models = {
    'logistic': LogisticRegression(class_weight='balanced', max_iter=1000),
    'random_forest': RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42),
    'xgboost': xgb.XGBClassifier(scale_pos_weight=len(y_train[y_train==0])/len(y_train[y_train==1]),
                                random_state=42)
}

# Entrenar y evaluar cada modelo
results = {}
for name, model in models.items():
    # Entrenamiento
    print(f"\nEntrenando {name}...")
    model.fit(X_train_scaled, y_train)
    
    # Validación cruzada
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='roc_auc')
    
    # Predicciones
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    # Guardar resultados
    results[name] = {
        'cv_scores': cv_scores,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'test_roc_auc': roc_auc
    }
    
    print(f"{name}:")
    print(f"CV ROC-AUC: {cv_scores.mean():.3f} (+/- {cv_scores.std()*2:.3f})")
    print(f"Test ROC-AUC: {roc_auc:.3f}")

# Crear directorio para modelos si no existe
import os
os.makedirs('../models', exist_ok=True)

# Guardar los modelos
for name, model in models.items():
    joblib.dump(model, f'../models/{name}_model.pkl')

print("\nModelos guardados en el directorio '../models'")


Entrenando logistic...
logistic:
CV ROC-AUC: 1.000 (+/- 0.000)
Test ROC-AUC: 1.000

Entrenando random_forest...
random_forest:
CV ROC-AUC: 1.000 (+/- 0.000)
Test ROC-AUC: 1.000

Entrenando xgboost...
xgboost:
CV ROC-AUC: 1.000 (+/- 0.000)
Test ROC-AUC: 1.000

Modelos guardados en el directorio '../models'
